# Import Libraries

In [76]:
import geemap
import os
import ee

# Create a interactive map

In [77]:
Map = geemap.Map(center=(18.423182, -93.1980127), zoom = 15)
# Add a basemap
Map.add_basemap('HYBRID')

Map

Map(center=[18.423182, -93.1980127], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggl…

# Create an area of interest (Roi)

In [78]:
# Create a Roi (Region of interst)

# Dos Bocas, Tabasco
roi = ee.Geometry.Polygon(
    [[[-93.21457597595585,18.412719723750588],
      [-93.18084451538456,18.412719723750588],
      [-93.18084451538456,18.4390217556039],
      [-93.21457597595585,18.4390217556039],
      [-93.21457597595585,18.412719723750588]]], None, False)


Map.addLayer(roi)

In [5]:
# Optionally, you can draw a roi in the map and use this seccion to
# save the roi

roi_draw = Map.draw_last_feature

# Create an image collection and the visualization parameters

In [79]:
# Create a cloud mask function
def Cloudmask(image):
# Exclude the pixels that represent clouds and cirrus on the image
# with the QA60 Band
    qa = image.select('QA60')
    cloud_type = 1 << 10
    cirrus_type = 1 << 11
    mask = qa.bitwiseAnd(cloud_type).eq(0) \
    .And(qa.bitwiseAnd(cirrus_type).eq(0))
    return image.updateMask(mask)

In [121]:
# Create an Image Collection
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
  .filterBounds(roi) \
  .filterDate('2018-07-01', '2021-04-30') \
  .filterMetadata ('CLOUDY_PIXEL_PERCENTAGE', 'Less_Than', 15) \
  .filterMetadata ('NODATA_PIXEL_PERCENTAGE', 'Less_Than', 70) \
  .map(Cloudmask)


In [122]:
# Verify the number of images
print(collection.size().getInfo())

52


In [123]:
# Get the dates in 'YYYY/MM/dd' for each image in the collection
dates =geemap.image_dates(collection, date_format='YYYY/MM/dd').getInfo()

print(dates)


KeyboardInterrupt: 

In [83]:
# Generate the visualization parameters
first_image = collection.first()

vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': [1, 1, 1]
}

Map.addLayer(first_image, vis, 'First image')

In [84]:
# Define arguments for animation function parameters.
video_args = {
  'dimensions': 900,
  'region': roi,
  'framesPerSecond': 2,
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': [1, 1, 1]
}

# Saving the result

In [89]:
# Save the result in "Downloads" folder
work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
# Create a name for the result    
out_gif = os.path.join(work_dir, "Sentinel-2_Dos_Bocas.gif")

In [90]:
# Download the result from the earth engine servers
geemap.download_ee_video(collection, video_args, out_gif)


Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\Juan Alexis\Downloads\Sentinel-2_Dos_Bocas.gif


# Add text to Gif

In [91]:
geemap.show_image(out_gif)

Output()

In [260]:
# Add dates to gif image
texted_gif = os.path.join(work_dir, "Sentinel-2_Dos_Bocas-txt.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=dates, font_size=30, font_color='#ffffff', add_progress_bar=False)

In [261]:
# Add a title
label = 'Refinería Dos Bocas, Tabasco'
geemap.add_text_to_gif(texted_gif, texted_gif, xy=('2%', '90%'), text_sequence=label, font_size=30, font_color='#ffffff', progress_bar_color='cyan', duration= 500)

In [262]:
# Show the results
geemap.show_image(texted_gif)

Output()